In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm
import logging
import unidecode
import math
import warnings

# Warnings na formatação do dataframe final 
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# === Configurações ===
GOOGLE_API_KEY = 'YOUR_API_KEY'
GOOGLE_PLACES_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
GOOGLE_PLACES_TEXT_URL = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
GOOGLE_GEOCODE_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

INPUT_FILE = 'CAMINHO_DO_CSV_A_SER_CHECADO'
FINAL_OUTPUT_FILE = 'CAMINHO_DO_CSV_FINAL'

# === Função para calcular distância entre duas coordenadas ===
def distancia_coordenadas(lat1, lon1, lat2, lon2):
    # Raio da Terra em quilômetros
    R = 6371.0

    # Converter graus para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Diferenças nas coordenadas
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    # Cálculo da distância usando a fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distância em quilômetros
    distance = R * c
    
    return distance

# === Função para geocodificar endereços ===
def geocodificacao_completa(address, city=None, country="Brasil"):
    """
    Converte um endereço em coordenadas geográficas usando a API de Geocodificação do Google
    """
    if pd.isna(address) or not str(address).strip():
        print(f"❌ Endereço vazio")
        return None, None
        
    # Formato completo do endereço para melhor precisão
    full_address = address
    if city and not pd.isna(city):
        full_address += f", {city}"
    full_address += f", {country}"
    
    print(f"🔍 Geocodificando: '{full_address}'")
    
    try:
        params = {
            'address': full_address,
            'key': GOOGLE_API_KEY
        }
        
        response = requests.get(GOOGLE_GEOCODE_URL, params=params, timeout=10)
        data = response.json()
        
        # Verifica se há erro na resposta
        if 'error_message' in data:
            print(f"❌ Erro na API de Geocodificação: {data['error_message']}")
            return None, None
            
        results = data.get('results', [])
        
        if not results:
            print(f"❌ Nenhum resultado encontrado para o endereço: '{full_address}'")
            return None, None
            
        # Pega o primeiro resultado (mais relevante)
        location = results[0]['geometry']['location']
        lat = location['lat']
        lng = location['lng']
        
        formatted_address = results[0].get('formatted_address', 'Endereço não formatado')
        print(f"✅ Endereço geocodificado: {formatted_address}")
        print(f"📍 Coordenadas: ({lat}, {lng})")
        
        return lat, lng
        
    except Exception as e:
        logging.error(f"Erro ao geocodificar endereço: {str(e)}")
        print(f"❌ Erro ao geocodificar: {str(e)}")
        return None, None

# === Busca POIs próximos que correspondem à (bandeira + rua) ===
def busca_poi(lat, lng, brand_name, street_name=None, radius=1000):
    if pd.isna(brand_name) or not str(brand_name).strip():
        print(f"❌ Nome da bandeira vazio para coordenadas ({lat}, {lng})")
        return None, False

    brand_name = str(brand_name).strip().lower()
    
    # Prepara a string de busca combinando bandeira e rua
    search_query = brand_name
    if street_name and not pd.isna(street_name) and str(street_name).strip():
        street = str(street_name).strip()
        search_query = f"{brand_name} {street}"
        print(f"🔍 Buscando por: '{search_query}'")
    
    try:
        # Primeira tentativa
        params = {
            'location': f'{lat},{lng}',
            'radius': radius,
            'keyword': search_query,  # Usando a combinação de bandeira + rua
            'key': GOOGLE_API_KEY
        }
        response = requests.get(GOOGLE_PLACES_URL, params=params, timeout=10)
        data = response.json()
        
        # Verifica algum possível erro na request
        if 'error_message' in data:
            print(f"❌ Erro na API Places: {data['error_message']}")
            return None, False
            
        results = data.get('results', [])

        # Segunda tentativa: tenta apenas com a bandeira
        if not results and street_name and not pd.isna(street_name):
            print(f"⚠️ Nenhum resultado com '{search_query}', tentando apenas com '{brand_name}'")
            params['keyword'] = brand_name
            response = requests.get(GOOGLE_PLACES_URL, params=params, timeout=10)
            data = response.json()
            results = data.get('results', [])

        # Terceira tentativa: tentativa a partir do tipo 'store' (loja)
        if not results:
            params = {
                'location': f'{lat},{lng}',
                'radius': radius,
                'type': 'store',  # Limita a lojas/mercados
                'key': GOOGLE_API_KEY
            }
            response = requests.get(GOOGLE_PLACES_URL, params=params, timeout=10)
            data = response.json()
            results = data.get('results', [])

        # Lista todos os POIs encontrados
        print(f"\n🔍 {len(results)} POIs próximos encontrados nas coordenadas ({lat}, {lng}):")
        match_found = False
        matching_place = None
        
        # Procura por correspondência com o nome da bandeira
        for i, poi in enumerate(results, 1):
            place_name = poi.get('name', 'Sem nome')
            place_address = poi.get('vicinity', 'Sem endereço')
            place_types = ", ".join(poi.get('types', []))
            
            # Exibe todos os POIs
            print(f"  {i}. {place_name} - {place_address}")
            
            # Verifica se o nome do POI contém o nome da bandeira
            if brand_name in place_name.lower() or brand_name.replace(" ", "") in place_name.lower():
                place_lat = poi['geometry']['location']['lat']
                place_lng = poi['geometry']['location']['lng']
                place_id = poi.get('place_id')  # Obtém o place_id
                
                # Formato do link que abre o painel lateral com as informações do POI
                place_maps_link = f"https://www.google.com/maps/place/?q=place_id:{place_id}"
                
                print(f"✅ ENCONTRADO: {place_name} - {place_address} ({place_types})")
                
                matching_place = {
                    'name': place_name,
                    'address': place_address,
                    'types': place_types,
                    'lat': place_lat,
                    'lng': place_lng,
                    'maps_link': place_maps_link,
                    'place_id': place_id,
                    'permanently_closed': poi.get('permanently_closed', False)
                }
                match_found = True
                break
        
        if match_found:
            return matching_place, True
        else:
            print(f"❌ Nenhum POI com nome '{brand_name}' encontrado próximo às coordenadas ({lat}, {lng})")
            return None, False
        
    # Verifica se houve erro na requisição        
    except Exception as e:
        logging.error(f"Erro ao buscar POIs próximos: {str(e)}")
        print(f"❌ Erro ao buscar POIs: {str(e)}")
        return None, False

# === Busca o POI pelo nome_unidade ===
def busca_nome_unidade(place_name, city=None, original_lat=None, original_lng=None):
    if pd.isna(place_name) or not str(place_name).strip():
        print(f"❌ Sem nome_unidade")
        return None, False

    place_name = str(place_name).strip()
    search_query = place_name
    
    # Adiciona a cidade se disponível para melhorar consulta
    if city and not pd.isna(city) and str(city).strip():
        search_query += f" {city}"
    
    try:
        # Faz a requisição para a API do Google Places
        params = {
            'query': search_query,
            'key': GOOGLE_API_KEY
        }
        print(f"\n🔍 Buscando '{search_query}' em qualquer localidade...")
        
        response = requests.get(GOOGLE_PLACES_TEXT_URL, params=params, timeout=10)
        data = response.json()
        
        # Verifica se houve erro na requisição 
        if 'error_message' in data:
            print(f"❌ Erro na API Places: {data['error_message']}")
            return None, False
            
        results = data.get('results', [])

        # Mostra resultados encontrados
        if results:
            print(f"\n📍 {len(results)} locais encontrados para '{search_query}':")
            
            # Se temos coordenadas originais, vamos ordenar por proximidade
            if original_lat is not None and original_lng is not None:
                # Calcular a distância de cada resultado para as coordenadas originais
                for poi in results:
                    poi_lat = poi['geometry']['location']['lat']
                    poi_lng = poi['geometry']['location']['lng']
                    poi['distance'] = distancia_coordenadas(original_lat, original_lng, poi_lat, poi_lng)
                
                # Ordenar resultados por distância
                results.sort(key=lambda x: x.get('distance', float('inf')))
                
                for i, poi in enumerate(results[:5], 1):
                    place_name = poi.get('name', 'Sem nome')
                    place_address = poi.get('formatted_address', 'Sem endereço')
                    distance = poi.get('distance', 'N/A')
                    print(f"  {i}. {place_name} - {place_address} (Distância: {distance:.2f} km)")
            
            # Pega o resultado mais próximo ou o primeiro se não tiver coordenadas originais
            best_match = results[0]
            
            place_name = best_match.get('name', 'Sem nome')
            place_address = best_match.get('formatted_address', 'Sem endereço')
            place_types = ", ".join(best_match.get('types', []))
            place_lat = best_match['geometry']['location']['lat']
            place_lng = best_match['geometry']['location']['lng']
            place_id = best_match.get('place_id')  # Obtém o place_id
            
            # Formato do link que abre o painel lateral com as informações do POI
            place_maps_link = f"https://www.google.com/maps/place/?q=place_id:{place_id}"
            
            distance_text = ""
            if original_lat is not None and original_lng is not None:
                distance = distancia_coordenadas(original_lat, original_lng, place_lat, place_lng)
                distance_text = f" (Distância: {distance:.2f} km)"
            
            print(f"✅ ENCONTRADO: {place_name} - {place_address}{distance_text}")
            
            matching_place = {
                'name': place_name,
                'address': place_address,
                'types': place_types,
                'lat': place_lat,
                'lng': place_lng,
                'maps_link': place_maps_link,
                'place_id': place_id,
                'permanently_closed': best_match.get('permanently_closed', False)
            }
            
            return matching_place, True
        else:
            print(f"❌ Nenhum local com nome '{search_query}' encontrado")
            return None, False
            
    except Exception as e:
        logging.error(f"Erro ao buscar local pelo nome: {str(e)}")
        print(f"❌ Erro ao buscar local: {str(e)}")
        return None, False

# === Processamento ===
df = pd.read_csv(INPUT_FILE, nrows=1)

# Adiciona colunas necessárias, mantendo apenas as relevantes para verificação de POIs
for col in ['poi_check', 'lat_poi', 'lon_poi', 'poi_nome', 'poi_endereco', 'poi_link',
            'lat_nome_unidade', 'lon_nome_unidade', 'poi_nome_unidade', 'poi_endereco_nome_unidade', 'poi_link_nome_unidade',
            'poi_encontrado', 'distancia_metros', 'poi_fechado_permanentemente', 
            'geocoded', 'lat_geocode', 'lng_geocode']:
    if col not in df.columns:
        df[col] = None

# Verifica se existem as colunas de coordenadas de referência
coord_columns = {
    'lat': next((col for col in ['latitude_ref', 'latitude', 'lat', 'latitude_coleta'] if col in df.columns), None),
    'lng': next((col for col in ['longitude_ref', 'longitude', 'lng', 'lon', 'longitude_coleta'] if col in df.columns), None)
}

# Identifica as colunas de endereços para geocodificação se necessário
address_columns = {
    'street': next((col for col in ['logradouro', 'endereco', 'address'] if col in df.columns), None),
    'city': next((col for col in ['municipio', 'cidade', 'city'] if col in df.columns), None)
}

needs_geocoding = False
if not coord_columns['lat'] or not coord_columns['lng']:
    if not address_columns['street']:
        raise ValueError("Sem coordenadas e sem coluna de endereço")
    else:
        needs_geocoding = True
        print(f"Usando a coluna {address_columns['street']} para geocodificação de endereços.")
else:
    print(f"Usando as colunas {coord_columns['lat']} e {coord_columns['lng']} como coordenadas de referência.")

# === Geocodifica endereços se necessário ===
geocoded_count = 0
if needs_geocoding:
    with tqdm(total=len(df), desc="Geocodificando endereços") as pbar:
        for idx in df.index:
            # Verifica se já tem coordenadas
            lat = df.at[idx, coord_columns['lat']] if coord_columns['lat'] else None
            lng = df.at[idx, coord_columns['lng']] if coord_columns['lng'] else None
            
            # Se não tiver coordenadas, tenta geocodificar o endereço
            if (pd.isna(lat) or pd.isna(lng)) and address_columns['street']:
                address = df.at[idx, address_columns['street']]
                city = df.at[idx, address_columns['city']] if address_columns['city'] else None
                
                print(f"\n🔍 Registro {idx} sem coordenadas. Tentando geocodificar endereço: '{address}'")
                geocoded_lat, geocoded_lng = geocodificacao_completa(address, city)
                
                if geocoded_lat and geocoded_lng:
                    # Armazena as coordenadas geocodificadas
                    df.at[idx, 'lat_geocode'] = geocoded_lat
                    df.at[idx, 'lng_geocode'] = geocoded_lng
                    df.at[idx, 'geocoded'] = 'SIM'
                    geocoded_count += 1
                    
                    # Se não houver colunas de coordenadas definidas, cria novas colunas
                    if not coord_columns['lat'] or not coord_columns['lng']:
                        df.at[idx, 'latitude'] = geocoded_lat
                        df.at[idx, 'longitude'] = geocoded_lng
                        if not coord_columns['lat']:
                            coord_columns['lat'] = 'latitude'
                        if not coord_columns['lng']:
                            coord_columns['lng'] = 'longitude'
                    else:
                        df.at[idx, coord_columns['lat']] = geocoded_lat
                        df.at[idx, coord_columns['lng']] = geocoded_lng
                        
                else:
                    df.at[idx, 'geocoded'] = 'NAO'
                    print(f"⚠️ Não foi possível geocodificar o endereço do registro {idx}")
            
            pbar.update(1)
            time.sleep(0.2)

    if geocoded_count > 0:
        print(f"\n✅ Geocodificados {geocoded_count} endereços com sucesso.")

# === Verifica POIs correspondentes à bandeira ou nome_unidade ===
with tqdm(total=len(df), desc="Verificando POIs") as pbar:
    for idx in df.index:
        # Prioriza coordenadas originais, mas usa geocodificadas se necessário
        lat = df.at[idx, coord_columns['lat']] if coord_columns['lat'] in df.columns else None
        lng = df.at[idx, coord_columns['lng']] if coord_columns['lng'] in df.columns else None
        
        # Usar coordenadas geocodificadas
        if (pd.isna(lat) or pd.isna(lng)) and not pd.isna(df.at[idx, 'lat_geocode']):
            lat = df.at[idx, 'lat_geocode']
            lng = df.at[idx, 'lng_geocode']
            print(f"\n🌐 Usando coordenadas geocodificadas para o registro {idx}: ({lat}, {lng})")
            
        brand = df.at[idx, 'bandeira'] if 'bandeira' in df.columns else None
        unit_name = df.at[idx, 'nome_unidade'] if 'nome_unidade' in df.columns else None
        city = df.at[idx, 'municipio'] if 'municipio' in df.columns else None
        street = df.at[idx, address_columns['street']] if address_columns['street'] else None

        if pd.isna(lat) or pd.isna(lng):
            df.at[idx, 'poi_check'] = 'NAO'
            print(f"\n⚠️ Coordenadas ausentes para o registro {idx} e não foi possível geocodificar")
            pbar.update(1)
            continue

        # Primeira tentativa: busca pela (bandeira + rua) próxima às coordenadas
        if brand is not None and not pd.isna(brand):
            print(f"\n🔎 Buscando POI com bandeira '{brand}' na rua '{street}' próximo às coordenadas ({lat}, {lng})")
            matching_place, found = busca_poi(lat, lng, brand, street)

            if found:
                df.at[idx, 'poi_check'] = 'SIM'
                df.at[idx, 'lat_poi'] = matching_place['lat']
                df.at[idx, 'lon_poi'] = matching_place['lng']
                df.at[idx, 'poi_nome'] = matching_place['name']
                df.at[idx, 'poi_endereco'] = matching_place['address']
                df.at[idx, 'poi_link'] = matching_place['maps_link']
                df.at[idx, 'poi_encontrado'] = matching_place['name']
                df.at[idx, 'poi_fechado_permanentemente'] = 'SIM' if matching_place.get('permanently_closed', False) else 'NAO'
                
                # Calcula a distância entre as coordenadas de referência e as do POI encontrado (em metros)
                distancia_km = distancia_coordenadas(lat, lng, matching_place['lat'], matching_place['lng'])
                df.at[idx, 'distancia_metros'] = int(distancia_km * 1000)  # Converte para metros
                
                pbar.update(1)
                continue
            else:
                df.at[idx, 'poi_check'] = 'NAO'
        else:
            df.at[idx, 'poi_check'] = 'NAO'
            
        # Segunda tentativa: busca pelo nome da unidade
        if unit_name is not None and not pd.isna(unit_name):
            print(f"\n🔎 Tentando com nome da unidade '{unit_name}' em qualquer localidade...")
            unit_place, unit_found = busca_nome_unidade(unit_name, city, lat, lng)
            
            if unit_found:
                df.at[idx, 'lat_nome_unidade'] = unit_place['lat']
                df.at[idx, 'lon_nome_unidade'] = unit_place['lng']
                df.at[idx, 'poi_nome_unidade'] = unit_place['name']
                df.at[idx, 'poi_endereco_nome_unidade'] = unit_place['address']
                df.at[idx, 'poi_link_nome_unidade'] = unit_place['maps_link']
                df.at[idx, 'poi_encontrado'] = unit_place['name']
                df.at[idx, 'poi_fechado_permanentemente'] = 'SIM' if unit_place.get('permanently_closed', False) else 'NAO'
                print(f"💡 Encontrado usando nome da unidade!")
                
                # Calcula a distância entre as coordenadas de referência e as do POI encontrado (em metros)
                distancia_km = distancia_coordenadas(lat, lng, unit_place['lat'], unit_place['lng'])
                df.at[idx, 'distancia_metros'] = int(distancia_km * 1000)  # Converte para metros e arredonda
        else:
            print(f"\n⚠️ Coluna 'nome_unidade' não encontrada ou valor ausente para o registro {idx}")

        pbar.update(1)
        time.sleep(0.2)

# === Organização do OUTPUT ===

# Combina resultados de busca por bandeira e por nome_unidade
df['lat'] = df['lat_poi'].fillna(df['lat_nome_unidade'])
df['lon'] = df['lon_poi'].fillna(df['lon_nome_unidade'])
df['nome'] = df['poi_nome'].fillna(df['poi_nome_unidade'])
df['endereco'] = df['poi_endereco'].fillna(df['poi_endereco_nome_unidade'])
df['link'] = df['poi_link'].fillna(df['poi_link_nome_unidade'])

# Define as colunas que serão mantidas no resultado final
columns_to_keep = [
    'bandeira', 'nome_unidade', 
    coord_columns['lat'], coord_columns['lng'], 
    'distancia_metros', 'lat', 'lon', 'nome', 'endereco', 'link'
]

if address_columns['street']:
    columns_to_keep.append(address_columns['street'])

# Filtra apenas as colunas existentes
available_columns = [col for col in columns_to_keep if col in df.columns]
df_final = df[available_columns]

# Remove "Brazil" dos endereços
if 'endereco' in df_final.columns:
    df_final['endereco'] = df_final['endereco'].str.replace(', Brazil', '', regex=False)

# Renomeia colunas para padronização
column_rename = {}

if coord_columns['lat'] in df_final.columns:
    column_rename[coord_columns['lat']] = 'latitude_coleta'
if coord_columns['lng'] in df_final.columns:
    column_rename[coord_columns['lng']] = 'longitude_coleta'
if 'lat' in df_final.columns:
    column_rename['lat'] = 'latitude_poi'
if 'lon' in df_final.columns:
    column_rename['lon'] = 'longitude_poi'
if 'nome' in df_final.columns:
    column_rename['nome'] = 'nome_poi'
if 'link' in df_final.columns:
    column_rename['link'] = 'poi_link'
if address_columns['street'] and address_columns['street'] in df_final.columns:
    column_rename[address_columns['street']] = 'endereco_coleta'

df_final.rename(columns=column_rename, inplace=True)

# Adiciona links para Google Maps com as coordenadas de coleta
if 'latitude_coleta' in df_final.columns and 'longitude_coleta' in df_final.columns:
    df_final['coleta_link'] = df_final.apply(
        lambda row: f"https://www.google.com/maps?q={row['latitude_coleta']},{row['longitude_coleta']}" 
        if not pd.isna(row.get('latitude_coleta')) and not pd.isna(row.get('longitude_coleta')) else None, 
        axis=1
    )

# Reorganiza as colunas para o resultado final
final_columns = ['bandeira', 'nome_unidade', 'endereco_coleta', 'latitude_coleta',
                'longitude_coleta', 'coleta_link', 'distancia_metros', 'latitude_poi',
                'longitude_poi', 'nome_poi', 'endereco', 'poi_link']

df_final = df_final[[col for col in final_columns if col in df_final.columns]]

# Salva o arquivo final
df_final.to_csv(FINAL_OUTPUT_FILE, index=False)

# === Estatísticas finais ===
poi_found_count = (df['poi_check'] == 'SIM').sum() if 'poi_check' in df.columns else 0
poi_alt_found_count = df['lat_nome_unidade'].notna().sum() if 'lat_nome_unidade' in df.columns else 0
geocoded_success = (df['geocoded'] == 'SIM').sum() if 'geocoded' in df.columns else 0
total_count = len(df)

print(f"\n📊 RESULTADOS:")
if needs_geocoding:
    print(f"🌐 Endereços geocodificados com sucesso: {geocoded_success}")
print(f"📍 Locais com POIs correspondentes à bandeira: {poi_found_count}")
print(f"📍 Locais com POIs correspondentes ao nome da unidade: {poi_alt_found_count}")
print(f"📊 Locais com POIs correspondentes à bandeira: {poi_found_count/total_count:.1%}")
print(f"📊 Locais com POIs correspondentes ao nome da unidade: {poi_alt_found_count/total_count:.1%}")
print(f"🏁 Total de registros processados: {total_count}")
print(f"\n✅ Arquivo final processado salvo em: {FINAL_OUTPUT_FILE}")